In [1]:
from pymatgen.io.jdftx.jdftxinfile import JDFTXInfile


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/richb/anaconda3/envs/dev3_atomate2/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/richb/anaconda3/envs/dev3_atomate2/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/richb/anaconda3/envs/dev3_atomate2/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 7

In [2]:
from pathlib import Path
from os import getcwd

ex_files_dir = Path(getcwd())/ "example_files"
ex_input_fname = ex_files_dir / "input-simple1.in"
print(Path(ex_input_fname).exists())
print(Path(ex_input_fname).parents[0])
# ex_outslice1 = []
# with open(ex_outslice_fname1) as f:
#     for line in f:
#         ex_outslice1.append(line)

joss = JDFTXInfile.from_file(ex_input_fname)

True
/Users/richb/vs/atomate2/tests/jdftx/io/example_files


In [4]:
print(joss)

here
lattice \
    10.000000000000  0.500000000000  0.000000000000 \
     0.000000000000 11.000000000000  0.000000000000 \
     0.000000000000  1.000000000000 12.000000000000  
ion C  0.500000000000  0.500000000000  0.600000000000 v  0.100000000000  0.200000000000  0.300000000000  0  
ion C  0.500000000000  0.500000000000  0.900000000000 v  0.100000000000  0.200000000000  0.300000000000  0  
ion O  0.200000000000  0.300000000000  0.400000000000 v  0.700000000000  0.800000000000  0.900000000000  1  
ion F  0.000000000000  0.010000000000  0.020000000000 v  0.990000000000  0.990000000000  0.990000000000  0  
ion C  0.100000000000  0.500000000000  0.600000000000 v  0.100000000000  0.200000000000  0.300000000000  0  

symmetry-matrix \
    1 0 0 \
    0 1 0 \
    0 0 1 \
     0.000000000000  0.000000000000  0.000000000000  
symmetry-matrix \
    1 0 0 \
    0 -1 0 \
    0 1 -1 \
     0.500000000000  0.500000000000  0.500000000000  

elec-cutoff 30.0 100.0  
elec-n-bands 34 
spintype no-spin

In [4]:
joss = JOutStructures.from_out_slice(ex_outslice1, iter_type="LatticeMinimize")

TypeError: Trajectory.__init__() missing 2 required positional arguments: 'species' and 'coords'